### ** Imports **

In [1]:
from file_io import load_image
from joblib import load
import numpy as np
import pandas as pd

### ** Auxiliary Function **

_ Inputs_

* __pred -__ the  binary segmentation;
* __gt -__ the ground truth; 
* __mask -__ the binary mask; 
* __ prob -__ the probability map.

_ Outputs _

* __sen -__ Sensibility;
* __spec -__ Specificity; 
* __acc -__ Accuracy; 
* __ auc -__ Area Under the ROC curve;
* __ mcc -__ Matthews Correlation Coefficient.

In [2]:
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef

def perform_metrics (pred, gt, mask, prob):
    
    # Suppressing background regions.
    pred = pred[mask>0]
    gt = gt[mask>0]
    prob = prob[mask>0]

    # Building confusion matrix.
    c_matrix = confusion_matrix(gt, pred)
    
    # Calculating ratios.
    tn = c_matrix[0,0]
    tp = c_matrix[1,1]
    fn = c_matrix[1,0]
    fp = c_matrix[0,1]
    
    # Finding the metrics.
    sen  = tp/(tp + fn)
    spec = tn/(tn + fp)
    acc  = (tp + tn)/(tp + tn + fp + fn)
    auc  = roc_auc_score (gt, prob)
    mcc  = matthews_corrcoef (gt, pred)
    
    
    return sen, spec, acc, auc, mcc

### ** Loading **

In [3]:
test_images = ['01R', '01L', '02R', '02L', '03R', '03L', '04R', '04L', '05R', '05L', '06R', '06L', '07R', '07L', '08R', '08L', '09R', '09L', '10R', '10L', '11R', '11L', '12R', '12L', '13R', '13L', '14R', '14L']
r_width, r_height = 999, 960

In [4]:
gts = np.zeros((len(test_images), r_height, r_width))
masks = np.zeros ((len(test_images), r_height, r_width))
segs = np.zeros((len(test_images), r_height, r_width))
probs = np.zeros((len(test_images), r_height, r_width))

In order for you to run the cell below, please download the annotations from [CHASE_DB1's website](https://blogs.kingston.ac.uk/retinal/chasedb1/) and change the paths accordingly.

In [5]:
#Change these paths
gts_path = '../resources/gts/CHASE_DB1/'

for i, image in enumerate(test_images):
    gt = load_image(gts_path + 'Image_%s_1stHO.png' %image, normalize = True)
    mask = load_image('../resources/masks/CHASE_DB1/mask_%s.png' %image)
    seg = load_image('../resources/binary_segmentations/CHASE_DB1/seg_%s.png' %image, normalize = True)
    prob = load('../resources/probability_maps/CHASE_DB1/prob_%s.npy' %image)[1]
    
    gts[i] = gt
    masks [i] = mask
    segs[i] = seg
    probs[i] = prob

### **Calculating**

In [6]:
sen_list, spec_list, acc_list, auc_list, mcc_list = [], [], [], [], []

for i in range (len(gts)):
    sen, spec, acc, auc, mcc = perform_metrics (segs[i], gts[i], masks[i], probs[i])
    sen_list.append(sen)
    spec_list.append(spec)
    acc_list.append(acc)
    auc_list.append(auc)
    mcc_list.append(mcc)

### ** Storing the Data**

In [7]:
data = {'Image' : test_images, 'Sensitivity': sen_list, 'Specificity': spec_list, 
        'Accuracy': acc_list, 'AUC': auc_list, 'MCC': mcc_list}

df = pd.DataFrame(data = data)
df = df[['Image', 'Sensitivity','Specificity','Accuracy', 'AUC', 'MCC']]
df.set_index('Image', inplace = True);

### **Image-level results **

In [8]:
df

,Sensitivity,Specificity,Accuracy,AUC,MCC
Image,,,,,
01R,0.750989,0.993156,0.966763,0.986760,0.818780
01L,0.828663,0.990718,0.974387,0.992642,0.854018
02R,0.764390,0.986515,0.961615,0.982528,0.798038
02L,0.745952,0.990786,0.962125,0.985061,0.806054
03R,0.786307,0.988889,0.966952,0.987492,0.821321
03L,0.801304,0.987156,0.966106,0.987092,0.825097
04R,0.733452,0.990193,0.961741,0.983259,0.793750
04L,0.760570,0.992236,0.965902,0.988831,0.821405
05R,0.777883,0.990670,0.964565,0.988474,0.827331


### **Average Results**

In [9]:
df.mean()

Sensitivity    0.777929
Specificity    0.986424
Accuracy       0.965332
AUC            0.985544
MCC            0.800887
dtype: float64